In [1]:
from haversine import haversine

from DB import *
db_connection = connect_db()

In [2]:
main = load_data_from_rds('생태통로', db_connection)
main

,번호,위도,경도,폭,연장,높이,개방도,차선_수,생태통로타입,등산객이용빈도,생태통로효율성
0,1495,35.36019,128.06375,5.00,45.0,NaN,NaN,4,0,1,3
1,1277,37.03408,128.40661,NaN,NaN,NaN,NaN,2,1,0,3
2,1278,36.89832,128.30563,NaN,NaN,NaN,NaN,2,1,0,1
3,1279,36.89687,128.30581,NaN,NaN,NaN,NaN,2,1,0,1
4,1090,36.71051,127.42222,8.23,38.0,NaN,NaN,6,0,5,1
...,...,...,...,...,...,...,...,...,...,...,...
492,1089,36.71278,127.42056,13.30,50.7,NaN,NaN,6,0,0,0
493,1087,36.71722,127.41778,13.60,38.7,NaN,NaN,4,0,5,0
494,1088,36.71222,127.42528,13.30,50.7,NaN,NaN,4,0,5,0
495,1091,36.76750,127.47333,7.00,20.0,NaN,NaN,2,0,0,0


In [3]:
trkroad = pd.read_csv('../data/variables/산책로_2013.csv', encoding='cp949')
trkroad

,좌표,산책로구분,명칭,코스번호,코스명,코스난이도,소요시간,코스길이(km)
0,"[[[127.7846719856947, 35.16237732803098], [127...",둘레길링크,하동호-삼화실,11코스,지리산둘레길,하,←4시간 9.3Km→,9342.428114
1,"[[[127.65931897718815, 35.18070376552526], [12...",둘레길링크,원부춘-가탄,15코스,지리산둘레길,상,←7시간30분 12.6Km→,13006.100118
2,"[[[127.75974281501351, 35.10290440337887], [12...",둘레길링크,하동읍-서당,13코스,지리산둘레길,하,←2시간30분 7.1Km→,7015.432513
3,"[[[127.58708082167367, 35.21776381053337], [12...",둘레길링크,목아재-당재,17코스,지리산둘레길,중,←3시간 7.8Km→,7909.008392
4,"[[[127.60337954723201, 35.42996623453473], [12...",둘레길링크,인월-금계,3코스,지리산둘레길,중,←8시간 19.3Km→,20457.857029
5,"[[[127.53082624461805, 35.440650005656984], [1...",둘레길링크,운봉-인월,2코스,지리산둘레길,중,←4시간 9.4Km→,9372.142179
6,"[[[127.4454921074571, 35.39082602777359], [127...",둘레길링크,주천-운봉,1코스,지리산둘레길,중,←6시간 14.3Km→,14564.613565
7,"[[[127.4635552642758, 35.26121956128309], [127...",둘레길링크,방광-산동,21코스,지리산둘레길,상,←6시간 13Km→,12770.776227
8,"[[[127.51500422178758, 35.205644472010604], [1...",둘레길링크,송정-오미,18코스,지리산둘레길,중,←5시간30분 9.2Km→,9875.170877
9,"[[[127.5728033822975, 35.20605879726299], [127...",둘레길링크,가탄-송정,16코스,지리산둘레길,상,←6시간30분 11.3Km→,9964.210622


In [4]:
# A-B 지점 간 거리(km) 계산 함수
def distance(A_lat, A_long, B_lat, B_long): # A 지점 위도 및 경도, B 지점 위도 및 경도
    A = (A_lat, A_long)
    B = (B_lat, B_long)
    dist = haversine(A, B, unit='km')
    return dist

In [11]:
dist_list = []
i_list = []
j_list = []
for i, m_row in main.iterrows(): # 각 생태통로 반복
    for j, tr_row in trkroad.iterrows(): # 각 산책로 반복
        for tr_coord in eval(tr_row['좌표']): # 각 산책로의 좌표 리스트
            for t in tr_coord :
              dist_list.append(distance(float(m_row['위도']), float(m_row['경도']), float(t[1]), float(t[0])))
              i_list.append(i)
              j_list.append(j)

In [12]:
dist_df = pd.DataFrame({'main_idx': i_list,
                        'trkroad_idx': j_list,
                        'distance': dist_list})
dist_df

,main_idx,trkroad_idx,distance
0,0,0,33.553785
1,0,0,33.539011
2,0,0,33.526624
3,0,0,33.521841
4,0,0,33.519236
...,...,...,...
4145969,496,9,161.896485
4145970,496,9,161.894991
4145971,496,9,161.877930
4145972,496,9,161.873315


## 최종 데이터

In [7]:
# 생태통로별 산책로까지의 최단 거리
df = pd.DataFrame({'번호': main['번호'], 
                   '산책로까지의 최단 거리(km)': dist_df.groupby(['main_idx'])['distance'].min()}).reset_index(drop=True)
df

,번호,산책로까지의 최단 거리(km)
0,1495,32.993820
1,1277,189.603501
2,1278,172.236247
3,1279,172.092457
4,1090,139.902024
...,...,...
492,1089,140.166888
493,1087,140.681388
494,1088,140.066912
495,1091,145.890079


## RDS에 데이터 업로드

In [8]:
table = load_data_from_rds('변수관계설정', db_connection)
table

,번호,등산로까지 최단거리(km),식생,경사도,지형기호(2.3km),주변 동물종 개수,주변동물 출현빈도,주변 로드킬 빈도
0,1001,1.903039,0,6,X,NaN,NaN,0
1,1002,0.120319,1,5,X,4.0,4.0,0
2,1004,0.047211,1,2,산지,1.0,1.0,0
3,1005,0.233820,1,2,X,4.0,4.0,0
4,1006,2.172428,0,4,하천,3.0,3.0,0
...,...,...,...,...,...,...,...,...
492,1561,0.000416,1,2,산지,3.0,5.0,4
493,1562,2.835246,1,5,산지,6.0,27.0,0
494,1564,0.886170,1,4,X,1.0,1.0,0
495,1567,9.125829,1,5,하천,4.0,5.0,0


In [9]:
table = table.merge(df, on='번호')
table

,번호,등산로까지 최단거리(km),식생,경사도,지형기호(2.3km),주변 동물종 개수,주변동물 출현빈도,주변 로드킬 빈도,산책로까지의 최단 거리(km)
0,1001,1.903039,0,6,X,NaN,NaN,0,237.040073
1,1002,0.120319,1,5,X,4.0,4.0,0,122.055124
2,1004,0.047211,1,2,산지,1.0,1.0,0,125.516218
3,1005,0.233820,1,2,X,4.0,4.0,0,128.701343
4,1006,2.172428,0,4,하천,3.0,3.0,0,103.572133
...,...,...,...,...,...,...,...,...,...
492,1561,0.000416,1,2,산지,3.0,5.0,4,219.634740
493,1562,2.835246,1,5,산지,6.0,27.0,0,38.432601
494,1564,0.886170,1,4,X,1.0,1.0,0,248.086515
495,1567,9.125829,1,5,하천,4.0,5.0,0,283.374921


In [10]:
upload_data_to_rds(table, '변수관계설정', db_connection)